In [1]:
from fastai2.text.all import *

In [2]:
path =  Path('/notebooks/storage/data/hate/data')
df = pd.read_json(path/'data.json')
df

,img,label,text,width,height,is_valid
42953,img/42953.png,0,its their character not their color that matters,400,265,False
23058,img/23058.png,0,don't be afraid to love again everyone is not like your ex,533,800,False
13894,img/13894.png,0,putting bows on your pet,800,558,False
37408,img/37408.png,0,i love everything and everybody! except for squirrels i hate squirrels,800,693,False
82403,img/82403.png,0,"everybody loves chocolate chip cookies, even hitler",416,550,False
...,...,...,...,...,...,...
83675,img/83675.png,0,i'm gonna be like phelps one day,366,550,True
37198,img/37198.png,0,when you're so relaxed you can feel yourself go to the next level,436,550,True
48670,img/48670.png,0,look at this sandwich maker club i found on wish on clearance,400,400,True
9863,img/09863.png,0,diverse group of women,549,825,True


In [21]:
test_df = pd.read_json(path/'test.jsonl', lines=True)

In [22]:
corpus ='\n'.join([x for x in df.text] + [x for x in test_df.text])
len(corpus)

620256

In [23]:
spacy = WordTokenizer()
toks = first(spacy([corpus]))
print(coll_repr(toks, 30))

(#138746) ['its','their','character','not','their','color','that','matters','\n','do',"n't",'be','afraid','to','love','again','everyone','is','not','like','your','ex','\n','putting','bows','on','your','pet','\n','i'...]


In [24]:
tkn = Tokenizer(spacy)
coll_repr(tkn(corpus), 30)

'(#139311) [\'xxbos\',\'its\',\'their\',\'character\',\'not\',\'their\',\'color\',\'that\',\'matters\',\'\\n\',\'do\',"n\'t",\'be\',\'afraid\',\'to\',\'love\',\'again\',\'everyone\',\'is\',\'not\',\'like\',\'your\',\'ex\',\'\\n\',\'putting\',\'bows\',\'on\',\'your\',\'pet\',\'\\n\'...]'

In [25]:
tokens = tkn(corpus)
tokens

(#139311) ['xxbos','its','their','character','not','their','color','that','matters','\n'...]

In [26]:
num = Numericalize()

num.setup([tkn(corpus)])

In [28]:
whole_vocab = num.vocab

In [9]:
nums = num(tokens)[:20]; nums

tensor([   2,  192,   79, 2379,   45,   79,  413,   32, 2380,    9,   29,   27,
          42, 1443,   14,  122,  194,  184,   18,   45])

In [10]:
' '.join(num.vocab[o] for o in nums)

"xxbos its their character not their color that matters \n do n't be afraid to love again everyone is not"

In [11]:
text_classifier_learner??

In [12]:
from fastai2.text.models.core import _model_meta
meta =_model_meta[AWD_LSTM]
model_path = untar_data(meta['url'], c_key='model')
fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
fnames

[Path('/storage/models/wt103-fwd/lstm_fwd.pth'),
 Path('/storage/models/wt103-fwd/itos_wt103.pkl')]

In [13]:
pretrained_vocab = fnames[1].load()
len(pretrained_vocab)

60000

In [14]:
missing = set(num.vocab) - set(pretrained_vocab)
len(missing)

240

In [29]:
dls = TextDataLoaders.from_df(df, path, vocab=whole_vocab, text_col='text',label_col='label', valid_col='is_valid' )

In [30]:
dls.show_batch()

,text,category
0,"xxbos housing , free gas , free xxunk , free healthcare and free education for my wives and children . to show xxunk for your xxunk , i 'll xxunk your 12 year old daughters , blow up your xxunk , xxunk and buses and xxunk hate through a xxunk xxunk in your local city and town xxunk with the purpose of xxunk your xxunk country into the same shithole i xxunk took xxunk from , allahu akbar !",1
1,"xxbos "" nobody is trying to take your guns ! "" "" we only want to make you xxunk them , xxunk xxunk , ban xxunk guns , xxunk magazine xxunk , xxunk xxunk them , ban or xxunk ammo , make other xxunk laws , and , if we catch you xxunk any of these made - up rules , throw you in prison … . at which point we will take your guns ! "" xxpad xxpad",0
2,"xxbos have you ever xxunk the history of the jews ? did you know that they have always xxunk together as a tribe , infiltrated xxunk , xxunk the xxunk xxunk of nations xxunk wars and xxunk created chaos in xxunk ? the jews have mass murdered millions of xxunk jews over the xxunk they xxunk seized control of the media so you 'll never find out study the history of the jews ! xxpad xxpad xxpad xxpad xxpad",1
3,"xxbos i had twins with my third husband , which i xxunk while married to my first husband , and xxunk while married to my second husband , who adopted them before i divorced him and married my third husband . but that was then , now i 'm married to my xxunk husband . i 'm just an xxunk who believes in the xxunk of marriage xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",1
4,"xxbos why do people think it 's okay to call babies he or she ? they ca n't speak yet so they ca n't say their xxunk gender please xxunk to them as xxunk or xxunk until they can say their xxunk xxunk otherwise you 're xxunk and transphobic well , there it is the xxunk thing i 'll read today xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",0
5,"xxbos what are they fighting over ? is it : a ) the last xxunk xxunk b ) he accidentally touch the homies hand xxunk a beer ? c ) xxunk used the last of the three flowers . d ) homie said "" i got you on gas "" but he only has $ 3 ! xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",1
6,"xxbos i think i 've got a gas leak , i woke up this morning feeling all light headed and xxunk , so just to be on the safe side , i asked the neighbours if they could come xxunk it out for me , i xxunk they 'd know , being jewish and all . xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",1
7,"xxbos in xxunk , the xxunk seized willie xxunk 's bank xxunk and real estate xxunk for a 16 million dollar tax xxunk . xxunk , al sharpton owes in xxunk of 19 million dollars in taxes and he xxunk the white house xxunk as a xxunk xxrep 3 ! explain white xxunk again ? xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",0
8,"xxbos i 'm a moderate muslim . that means that while other muslims are putting bombs on your buses and raping your children i 'll be telling you that islam is peaceful , protesting when you insult my religion and xxunk your government to take away your free speech . a moderate muslim xxunk ! xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",0


In [31]:
len(dls.train_ds), len(dls.valid_ds)

(8500, 500)

In [88]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.8, metrics=[accuracy, RocAucBinary()])


In [89]:
learn.fine_tune(20, 1e-2)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,0.848293,0.708026,0.544000,0.554544,00:04


epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,0.719061,0.720450,0.538000,0.583344,00:07
1,0.667868,0.708315,0.548000,0.604616,00:07
2,0.635200,0.756339,0.548000,0.583320,00:07
3,0.611655,0.771287,0.526000,0.595088,00:07
4,0.595387,0.749637,0.538000,0.597064,00:07
5,0.582934,0.801127,0.534000,0.607528,00:07
6,0.569016,0.785077,0.542000,0.615696,00:07
7,0.548179,0.884942,0.550000,0.616552,00:07
8,0.534648,0.879437,0.558000,0.617400,00:07
9,0.530502,0.836562,0.540000,0.620240,00:07


In [34]:
def gen_submit(learn, fname):
    test_df = pd.read_json(path/'test.jsonl', lines=True)
    test_dl = learn.dls.test_dl(test_df)
    preds = learn.get_preds(dl=test_dl)[0]
    _, indcs = preds.max(dim=1)
    probs= preds[:,1]
    submit_df = pd.DataFrame()
    submit_df['id'] = test_df['id']
    submit_df['proba'] = probs
    submit_df['label'] = indcs
    submit_df = submit_df.set_index('id')
    submit_df.to_csv(fname, header=True)

In [35]:
gen_submit(learn, 'attempt3.csc')

In [73]:
preds, labels = learn.get_preds(dl=learn.dls.valid)

In [74]:
accuracy(preds, labels)

tensor(0.5360)

In [75]:
_, pred_lab =preds.max(dim=1)

In [76]:
sum(pred_lab==labels).float() / len(preds)

tensor(0.5360)

In [77]:
import sklearn
sklearn.metrics.roc_auc_score(labels, preds[:,1])

0.611736